 Basic Modules

In [4]:
# Standard Library
import io
import wave
from typing import Tuple, Union

# Audio Processing
import librosa
import librosa.display
import noisereduce as nr
import pyaudio
import sounddevice as sd
import soundfile as sf
import torchaudio

# Numerical Computing
import numpy as np
import torch

# Audio Effects
from pedalboard import Pedalboard, NoiseGate, Compressor, LowShelfFilter, Gain

# IPython/Jupyter (commented out as not needed in production)
from IPython.display import Audio, display
# import matplotlib.pyplot as plt
# from pathlib import Path

In [5]:
import sys
import os
data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

In [6]:
audio_folder=data_path+r"\audio"

In [7]:
input_audio_path = audio_folder+r'\audio1.wav'

Loading the Audio File

In [8]:
# Load the audio file
input_audio, sr = torchaudio.load(input_audio_path)
print(input_audio.shape)
print(sr)

torch.Size([2, 462180])
44100


In [9]:
display(Audio(str(input_audio_path), rate=sr))

Preprocessing the Audio Pipeline

In [10]:
def detect_audio_input(
    input_data: Union[str, torch.Tensor, np.ndarray, tuple, "pyaudio.PyAudio", "librosa.Audio"],
    sr: int = None
) -> Tuple[np.ndarray, int]:
    """
    Detects and standardizes audio input from:
    - File paths (WAV, MP3, etc.)
    - PyTorch tensors
    - NumPy arrays
    - Librosa-loaded audio
    - SoundFile-loaded audio
    - PyAudio streams (raw bytes or buffers)

    Args:
        input_data: Input audio (file path, tensor, array, librosa tuple, PyAudio stream)
        sr: Sample rate (required for raw arrays/tensors)

    Returns:
        Tuple of (audio_array, sample_rate) where audio_array is shaped (channels, samples)

    Raises:
        ValueError: If input format is invalid or unsupported
    """
    print(f"\n[DEBUG] Input type: {type(input_data)}")

    # Case 1: File path (string)
    if isinstance(input_data, str):
        print(f"[DEBUG] Detected file path: {input_data}")
        if not os.path.exists(input_data):
            raise FileNotFoundError(f"File not found: {input_data}")

        # Try loading with different backends
        try:
            print("[DEBUG] Trying torchaudio...")
            audio, sr = torchaudio.load(input_data, normalize=True)
            print(f"[DEBUG] Loaded with torchaudio. Shape: {audio.shape}, SR: {sr}")
            return audio.numpy(), sr
        except Exception as e:
            print(f"[DEBUG] Torchaudio failed: {str(e)}")
            try:
                print("[DEBUG] Trying librosa...")
                audio, sr = librosa.load(input_data, sr=sr, mono=False)
                print(f"[DEBUG] Loaded with librosa. Shape: {audio.shape}, SR: {sr}")
                if audio.ndim == 1:
                    audio = audio[np.newaxis, :]  # (1, N) for mono
                return audio, sr
            except Exception as e:
                print(f"[DEBUG] Librosa failed: {str(e)}")
                try:
                    print("[DEBUG] Trying soundfile...")
                    audio, sr = sf.read(input_data)
                    print(f"[DEBUG] Loaded with soundfile. Shape: {audio.shape}, SR: {sr}")
                    if audio.ndim == 1:
                        audio = audio[np.newaxis, :]
                    else:
                        audio = audio.T  # (channels, samples)
                    return audio, sr
                except Exception as e:
                    raise ValueError(f"Failed to load file: {str(e)}")

    # Case 2: PyTorch tensor
    elif isinstance(input_data, torch.Tensor):
        print(f"[DEBUG] Detected PyTorch tensor. Shape: {input_data.shape}")
        if sr is None:
            raise ValueError("Sample rate (sr) must be provided for tensors")
        audio = input_data.numpy()
        if audio.ndim == 1:
            audio = audio[np.newaxis, :]  # (1, N)
        return audio, sr

    # Case 3: NumPy array
    elif isinstance(input_data, np.ndarray):
        print(f"[DEBUG] Detected NumPy array. Shape: {input_data.shape}")
        if sr is None:
            raise ValueError("Sample rate (sr) must be provided for arrays")
        if input_data.ndim == 1:
            input_data = input_data[np.newaxis, :]  # (1, N)
        return input_data, sr

    # Case 4: Librosa-style tuple (audio, sr)
    elif isinstance(input_data, tuple) and len(input_data) == 2:
        print("[DEBUG] Detected librosa-style (audio, sr) tuple")
        audio, sr = input_data
        if isinstance(audio, np.ndarray):
            if audio.ndim == 1:
                audio = audio[np.newaxis, :]  # (1, N)
            return audio, sr
        raise ValueError("Invalid librosa tuple format")

    # Case 5: PyAudio stream (bytes or buffer)
    elif isinstance(input_data, (bytes, io.BytesIO)):
        print("[DEBUG] Detected PyAudio stream (bytes/BytesIO)")
        if sr is None:
            raise ValueError("Sample rate (sr) must be provided for raw audio streams")
        try:
            # Convert bytes to numpy array (assumes 16-bit PCM)
            audio = np.frombuffer(input_data, dtype=np.int16)
            audio = audio.astype(np.float32) / 32768.0  # Normalize to [-1, 1]
            audio = audio[np.newaxis, :]  # (1, N)
            return audio, sr
        except Exception as e:
            raise ValueError(f"Failed to decode PyAudio stream: {str(e)}")

    # Case 6: SoundFile object (if passed directly)
    elif hasattr(input_data, 'read') and hasattr(input_data, 'samplerate'):
        print("[DEBUG] Detected SoundFile object")
        try:
            audio = input_data.read()
            sr = input_data.samplerate
            if audio.ndim == 1:
                audio = audio[np.newaxis, :]
            else:
                audio = audio.T  # (channels, samples)
            return audio, sr
        except Exception as e:
            raise ValueError(f"Failed to read SoundFile object: {str(e)}")

    else:
        raise ValueError(f"Unsupported input type: {type(input_data)}")


In [11]:

print("=== Testing Detection ===")

# Test with file path
try:
        print("\nTest 1: File path")
        audio, sr = detect_audio_input(input_audio_path)  # Change to real file
        print(f"Success! Shape: {audio.shape}, SR: {sr}")
except Exception as e:
        print(f"File test failed: {str(e)}")

    # Test with PyTorch tensor
try:
        print("\nTest 2: PyTorch tensor")
        test_tensor = torch.randn(2, 16000)  # 2 channels, 16000 samples
        audio, sr = detect_audio_input(test_tensor, sr=16000)
        print(f"Success! Shape: {audio.shape}, SR: {sr}")
except Exception as e:
        print(f"Tensor test failed: {str(e)}")

    # Test with NumPy array
try:
        print("\nTest 3: NumPy array")
        test_array = np.random.randn(44100)
        audio, sr = detect_audio_input(test_array, sr=44100)
        print(f"Success! Shape: {audio.shape}, SR: {sr}")
except Exception as e:
        print(f"Array test failed: {str(e)}")

# Test with Librosa
print("\n=== Testing Librosa Input ===")
librosa_audio, librosa_sr = librosa.load(input_audio_path, sr=None, mono=False)
detected_audio, detected_sr = detect_audio_input((librosa_audio, librosa_sr))
print(f"Librosa test: Output shape={detected_audio.shape}, SR={detected_sr}")

# Test with PyAudio (simulated bytes)
print("\n=== Testing PyAudio Input ===")
pyaudio_bytes = np.random.randint(-32768, 32767, 44100, dtype=np.int16).tobytes()
detected_audio, detected_sr = detect_audio_input(pyaudio_bytes, sr=44100)
print(f"PyAudio test: Output shape={detected_audio.shape}, SR={detected_sr}")

# Test with SoundFile
print("\n=== Testing SoundFile Input ===")
with sf.SoundFile(input_audio_path) as sf_file:
        detected_audio, detected_sr = detect_audio_input(sf_file)
print(f"SoundFile test: Output shape={detected_audio.shape}, SR={detected_sr}")

=== Testing Detection ===

Test 1: File path

[DEBUG] Input type: <class 'str'>
[DEBUG] Detected file path: d:\PERSONAL\GitHub\College\MOTIVE\backend\data\audio\audio1.wav
[DEBUG] Trying torchaudio...
[DEBUG] Loaded with torchaudio. Shape: torch.Size([2, 462180]), SR: 44100
Success! Shape: (2, 462180), SR: 44100

Test 2: PyTorch tensor

[DEBUG] Input type: <class 'torch.Tensor'>
[DEBUG] Detected PyTorch tensor. Shape: torch.Size([2, 16000])
Success! Shape: (2, 16000), SR: 16000

Test 3: NumPy array

[DEBUG] Input type: <class 'numpy.ndarray'>
[DEBUG] Detected NumPy array. Shape: (44100,)
Success! Shape: (1, 44100), SR: 44100

=== Testing Librosa Input ===

[DEBUG] Input type: <class 'tuple'>
[DEBUG] Detected librosa-style (audio, sr) tuple
Librosa test: Output shape=(2, 462180), SR=44100

=== Testing PyAudio Input ===

[DEBUG] Input type: <class 'bytes'>
[DEBUG] Detected PyAudio stream (bytes/BytesIO)
PyAudio test: Output shape=(1, 44100), SR=44100

=== Testing SoundFile Input ===

[DE

In [12]:
def audio_to_tensor(
    input_data: Union[str, torch.Tensor, np.ndarray, tuple, bytes, io.BytesIO],
    sr: int = None,
    device: str = "cpu",
    normalize: bool = True,
    verbose: bool = True
) -> Tuple[torch.Tensor, int]:
    """
    Enhanced audio to tensor converter with detailed debugging.

    Args:
        input_data: Audio input (file path, tensor, array, librosa tuple, PyAudio bytes)
        sr: Sample rate (required for raw arrays/tensors)
        device: Target device ("cpu" or "cuda")
        normalize: Normalize to [-1, 1] for float tensors
        verbose: Enable detailed debug prints

    Returns:
        Tuple of (audio_tensor, sample_rate) with shape (channels, samples)
    """
    def debug_print(msg):
        if verbose:
            print(f"[DEBUG] {msg}")

    debug_print(f"Input type: {type(input_data)}")

    try:
        # Standardize input to numpy array + sample rate
        audio_np, sr = detect_audio_input(input_data, sr)
        debug_print(f"Standardized numpy array. Shape: {audio_np.shape}, SR: {sr}")

        # Convert to tensor
        audio_tensor = torch.from_numpy(audio_np).to(device)
        debug_print(f"Raw tensor. Shape: {audio_tensor.shape}, dtype: {audio_tensor.dtype}")

        # Normalization handling
        if normalize:
            if audio_tensor.is_floating_point():
                max_val = torch.max(torch.abs(audio_tensor))
                if max_val > 1.0 or torch.min(audio_tensor) < -1.0:
                    audio_tensor = audio_tensor / max_val
                    debug_print(f"Normalized to [-1, 1]. Max before: {max_val:.4f}")
                else:
                    debug_print("Already in [-1, 1] range, skipping normalization")
            else:
                debug_print("Integer tensor detected, skipping normalization")

        debug_print(f"Final tensor. Shape: {audio_tensor.shape}, dtype: {audio_tensor.dtype}")
        return audio_tensor, sr

    except Exception as e:
        debug_print(f"Conversion failed: {str(e)}")
        raise ValueError(f"Audio to tensor conversion error: {str(e)}")

In [13]:
# Test cases matching your console output
print("\n=== Librosa Tuple Test ===")
librosa_audio = np.random.randn(2, 462180).astype(np.float32)
tensor, sr = audio_to_tensor((librosa_audio, 44100))

print("\n=== PyAudio Bytes Test ===")
pyaudio_bytes = np.random.randint(-32768, 32767, 44100,
                                     dtype=np.int16).tobytes()
tensor, sr = audio_to_tensor(pyaudio_bytes, sr=44100)

print("\n=== File Path Test ===")
tensor, sr = audio_to_tensor(input_audio_path)  # Will show torchaudio loading


=== Librosa Tuple Test ===
[DEBUG] Input type: <class 'tuple'>

[DEBUG] Input type: <class 'tuple'>
[DEBUG] Detected librosa-style (audio, sr) tuple
[DEBUG] Standardized numpy array. Shape: (2, 462180), SR: 44100
[DEBUG] Raw tensor. Shape: torch.Size([2, 462180]), dtype: torch.float32
[DEBUG] Normalized to [-1, 1]. Max before: 4.7327
[DEBUG] Final tensor. Shape: torch.Size([2, 462180]), dtype: torch.float32

=== PyAudio Bytes Test ===
[DEBUG] Input type: <class 'bytes'>

[DEBUG] Input type: <class 'bytes'>
[DEBUG] Detected PyAudio stream (bytes/BytesIO)
[DEBUG] Standardized numpy array. Shape: (1, 44100), SR: 44100
[DEBUG] Raw tensor. Shape: torch.Size([1, 44100]), dtype: torch.float32
[DEBUG] Already in [-1, 1] range, skipping normalization
[DEBUG] Final tensor. Shape: torch.Size([1, 44100]), dtype: torch.float32

=== File Path Test ===
[DEBUG] Input type: <class 'str'>

[DEBUG] Input type: <class 'str'>
[DEBUG] Detected file path: d:\PERSONAL\GitHub\College\MOTIVE\backend\data\audio

In [14]:
def process_audio_input(
    input_data: Union[str, torch.Tensor, np.ndarray, tuple, bytes, io.BytesIO],
    sr: int = None,
    device: str = "cpu",
    normalize: bool = True,
    verbose: bool = True
) -> Tuple[torch.Tensor, int]:
    """
    Universal audio input processor that automatically handles conversion to tensor.

    Args:
        input_data: Audio input (file path, tensor, array, librosa tuple, bytes)
        sr: Sample rate (required for non-file inputs)
        device: Target device ("cpu" or "cuda")
        normalize: Whether to normalize audio to [-1, 1]
        verbose: Enable debug printing

    Returns:
        Tuple of (audio_tensor, sample_rate)
    """
    def debug_print(msg):
        if verbose:
            print(f"[DEBUG] {msg}")

    debug_print(f"Input type detected: {type(input_data)}")

    # Case 1: Already a PyTorch tensor
    if isinstance(input_data, torch.Tensor):
        debug_print("Input is already a PyTorch tensor")
        audio_tensor = input_data.to(device)

        # Validate tensor shape
        if audio_tensor.ndim == 1:
            debug_print("Adding channel dimension to mono audio")
            audio_tensor = audio_tensor.unsqueeze(0)  # (1, N)
        elif audio_tensor.ndim > 2:
            raise ValueError(f"Invalid tensor shape: {audio_tensor.shape}. Expected (channels, samples)")

        # Normalize if needed
        if normalize and audio_tensor.is_floating_point():
            max_val = torch.max(torch.abs(audio_tensor))
            if max_val > 1.0:
                audio_tensor = audio_tensor / max_val
                debug_print(f"Normalized tensor to [-1, 1]. Max before: {max_val:.4f}")

        if sr is None:
            raise ValueError("Sample rate must be provided for tensor input")

        return audio_tensor, sr

    # Case 2: Not a tensor - convert using audio_to_tensor
    else:
        debug_print("Input is not a PyTorch tensor - converting...")
        try:
            return audio_to_tensor(
                input_data=input_data,
                sr=sr,
                device=device,
                normalize=normalize,
                verbose=verbose
            )
        except Exception as e:
            raise ValueError(f"Failed to convert input to tensor: {str(e)}")

In [15]:
# Test with various input types
test_sr = 44100
# 1. Test with PyTorch tensor (should pass through directly)
print("\n=== Test 1: PyTorch tensor input ===")
tensor_input = torch.rand(2, 44100)  # Stereo audio
processed, sr = process_audio_input(tensor_input, sr=test_sr)
print(f"Output shape: {processed.shape}, SR: {sr}")
# 2. Test with file path (should convert)
print("\n=== Test 2: File path input ===")
file_input = "audio.wav"  # Replace with real file
try:
    processed, sr = process_audio_input(file_input)
    print(f"Output shape: {processed.shape}, SR: {sr}")
except Exception as e:
    print(f"Error: {str(e)}")
# 3. Test with numpy array (should convert)
print("\n=== Test 3: NumPy array input ===")
numpy_input = np.random.randn(44100)  # Mono audio
processed, sr = process_audio_input(numpy_input, sr=test_sr)
print(f"Output shape: {processed.shape}, SR: {sr}")


=== Test 1: PyTorch tensor input ===
[DEBUG] Input type detected: <class 'torch.Tensor'>
[DEBUG] Input is already a PyTorch tensor
Output shape: torch.Size([2, 44100]), SR: 44100

=== Test 2: File path input ===
[DEBUG] Input type detected: <class 'str'>
[DEBUG] Input is not a PyTorch tensor - converting...
[DEBUG] Input type: <class 'str'>

[DEBUG] Input type: <class 'str'>
[DEBUG] Detected file path: audio.wav
[DEBUG] Conversion failed: File not found: audio.wav
Error: Failed to convert input to tensor: Audio to tensor conversion error: File not found: audio.wav

=== Test 3: NumPy array input ===
[DEBUG] Input type detected: <class 'numpy.ndarray'>
[DEBUG] Input is not a PyTorch tensor - converting...
[DEBUG] Input type: <class 'numpy.ndarray'>

[DEBUG] Input type: <class 'numpy.ndarray'>
[DEBUG] Detected NumPy array. Shape: (44100,)
[DEBUG] Standardized numpy array. Shape: (1, 44100), SR: 44100
[DEBUG] Raw tensor. Shape: torch.Size([1, 44100]), dtype: torch.float64
[DEBUG] Normaliz

In [16]:
def play_tensor_audio(
    audio_tensor: torch.Tensor,
    sample_rate: int,
    device: int = None,
    blocking: bool = False,
    normalize: bool = True
):
    """
    Enhanced audio playback function with device selection.

    Args:
        audio_tensor: Audio tensor (shape: [channels, samples])
        sample_rate: Playback sample rate in Hz
        device: Output device ID (None for default)
        blocking: Whether to wait for playback completion
        normalize: Whether to normalize before playback
    """
    print("\n[PLAYBACK] Preparing audio playback")

    # Convert to numpy
    audio_np = audio_tensor.cpu().numpy()
    print(f"  - Input shape: {audio_np.shape}")

    # Handle mono/stereo
    if audio_np.ndim == 1:
        audio_np = np.expand_dims(audio_np, axis=0)
        print("  - Expanded mono to stereo")
    elif audio_np.ndim > 2:
        raise ValueError("Audio must be 1D (mono) or 2D (stereo)")

    # Normalize if requested
    if normalize:
        max_val = np.max(np.abs(audio_np))
        if max_val > 1.0:
            audio_np = audio_np / max_val
            print(f"  - Normalized (max before: {max_val:.4f})")

    # Device info
    if device is not None:
        dev_info = sd.query_devices(device)
        print(f"  - Output device: {dev_info['name']}")

    # Play audio
    try:
        print("  - Starting playback...")
        sd.play(
            audio_np.T,  # sounddevice expects (samples, channels)
            samplerate=sample_rate,
            device=device,
            blocking=blocking
        )

        if not blocking:
            print("  - Playback started (non-blocking)")
        else:
            print("  - Playback completed")
    except Exception as e:
        raise RuntimeError(f"Playback failed: {str(e)}")


In [17]:
play_tensor_audio(input_audio, sr)


[PLAYBACK] Preparing audio playback
  - Input shape: (2, 462180)
  - Starting playback...
  - Playback started (non-blocking)


In [18]:
#type: ignore
def enhance_audio_tensor(audio: torch.Tensor, sample_rate: int) -> Tuple[torch.Tensor, int]:
    """
    Enhance audio that's already loaded as a PyTorch tensor.

    Args:
        audio: PyTorch tensor of shape (channels, samples)
        sample_rate: Audio sample rate in Hz

    Returns:
        Tuple of (enhanced_audio_tensor, sample_rate)
    """
    # Convert to numpy array for processing
    audio_np = audio.numpy()

    # Apply noise reduction
    reduced_noise = nr.reduce_noise(
        y=audio_np,
        sr=sample_rate,
        stationary=True,
        prop_decrease=0.75
    )

    # Apply effects using Pedalboard
    board = Pedalboard([
        NoiseGate(threshold_db=-30, ratio=1.5, release_ms=250),
        Compressor(threshold_db=-16, ratio=2.5),
        LowShelfFilter(cutoff_frequency_hz=400, gain_db=10, q=1),
        Gain(gain_db=10)
    ])

    # Process audio
    enhanced_np = board(reduced_noise, sample_rate)

    # Convert back to PyTorch tensor
    enhanced_audio = torch.from_numpy(enhanced_np)

    return enhanced_audio, sample_rate

In [19]:
enhanced_audio, sr = enhance_audio_tensor(input_audio, sr)

In [20]:
print(enhanced_audio.shape)
print(sr)
print(enhanced_audio.dtype)
print(type(enhanced_audio))

torch.Size([2, 462180])
44100
torch.float32
<class 'torch.Tensor'>


In [21]:
def save_enhanced_audio(
    audio: torch.Tensor,
    sample_rate: int,
    output_path: str
) -> None:
    """
    Save enhanced audio tensor to file

    Args:
        audio: Enhanced audio tensor
        sample_rate: Audio sample rate
        output_path: Path to save the enhanced audio
    """
    torchaudio.save(output_path, audio, sample_rate)
    print(f"Enhanced audio saved to: {output_path}")


In [22]:
enhace_audio_path = audio_folder+r"\enhanced_audio.wav"
save_enhanced_audio(enhanced_audio, sr, enhace_audio_path)

Enhanced audio saved to: d:\PERSONAL\GitHub\College\MOTIVE\backend\data\audio\enhanced_audio.wav


In [23]:
play_tensor_audio(enhanced_audio, sr)


[PLAYBACK] Preparing audio playback
  - Input shape: (2, 462180)
  - Normalized (max before: 1.5120)
  - Starting playback...
  - Playback started (non-blocking)


In [24]:
print("\n=== Audio Enhancement Pipeline ===")

# Configuration
data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))
audio_folder = os.path.join(data_path, "audio")
input_path = os.path.join(audio_folder, 'audio1.wav')
output_path = os.path.join(audio_folder, "enhanced_audio.wav")
try:
            # Load audio from other modules
            print("\n[STAGE 1] Loading audio...")
            input_audio, sr = librosa.load(input_audio_path, sr=None, mono=False)
            # input_audio, sr = torchaudio.load(input_audio_path)
            print(f"  - Loaded audio: {input_audio.shape}, {sr}Hz")
            # stage 2 checking audio
            if torch.is_tensor(input_audio):
                pass
            else:
                # Process audio
                input_audio, sr = process_audio_input(input_audio, sr)
            # Play original
            print("\n[STAGE 2] Playing original audio...")
            play_tensor_audio(input_audio, sr,blocking=True)

            # Enhance audio
            print("\n[STAGE 3] Enhancing audio...")
            enhanced_audio, sr = enhance_audio_tensor(input_audio, sr)
            print(f"  - Enhanced audio: {enhanced_audio.shape}, {sr}Hz")

            # Save enhanced
            print("\n[STAGE 4] Saving enhanced audio...")
            save_enhanced_audio(enhanced_audio, sr, output_path)

            # Play enhanced
            print("\n[STAGE 5] Playing enhanced audio...")
            play_tensor_audio(enhanced_audio, sr, blocking=True)

            print("\n[SUCCESS] Pipeline completed successfully")
except Exception as e:
            print(f"\n[ERROR] Pipeline failed: {str(e)}")
            raise


=== Audio Enhancement Pipeline ===

[STAGE 1] Loading audio...
  - Loaded audio: (2, 462180), 44100Hz
[DEBUG] Input type detected: <class 'numpy.ndarray'>
[DEBUG] Input is not a PyTorch tensor - converting...
[DEBUG] Input type: <class 'numpy.ndarray'>

[DEBUG] Input type: <class 'numpy.ndarray'>
[DEBUG] Detected NumPy array. Shape: (2, 462180)
[DEBUG] Standardized numpy array. Shape: (2, 462180), SR: 44100
[DEBUG] Raw tensor. Shape: torch.Size([2, 462180]), dtype: torch.float32
[DEBUG] Already in [-1, 1] range, skipping normalization
[DEBUG] Final tensor. Shape: torch.Size([2, 462180]), dtype: torch.float32

[STAGE 2] Playing original audio...

[PLAYBACK] Preparing audio playback
  - Input shape: (2, 462180)
  - Starting playback...
  - Playback completed

[STAGE 3] Enhancing audio...
  - Enhanced audio: torch.Size([2, 462180]), 44100Hz

[STAGE 4] Saving enhanced audio...
Enhanced audio saved to: d:\PERSONAL\GitHub\College\MOTIVE\backend\data\audio\enhanced_audio.wav

[STAGE 5] Playi